## Import Libraries

In [1]:
import json
import uuid
import io
import copy
import re

In [2]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimlab.data_profile.' + cim_profile)
sparql = importlib.import_module('cimlab.loaders.sparql.' + cim_profile)

In [3]:
from cimgraph.loaders import Parameter, ConnectionParameters
from cimgraph.loaders.blazegraph.blazegraph import BlazegraphConnection
from cimgraph.models.model_parsers import add_to_catalog, add_to_typed_catalog, cim_dump, item_dump

from cimgraph.models import NewModel

In [4]:
# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params, 'rc4_2021')

## Create New Feeder Model

In [5]:
reg = cim.GeographicalRegion(name = 'demo_region', mRID = str(uuid.uuid4()))
subreg = cim.SubGeographicalRegion(name = 'demo_subregion', mRID = str(uuid.uuid4()), Region=reg)
sub = cim.Substation(name = 'demo_sub', mRID = str(uuid.uuid4()), Region=subreg)
fdr = cim.Feeder(name = 'demo_feeder', mRID = str(uuid.uuid4()), NormalEnergizingSubstation=sub)

In [6]:
network = NewModel(bg)
network.Feeder = fdr
network.add_to_typed_catalog([reg, subreg, sub, fdr])

In [7]:
def create_line(network, name:str, bus1:str, bus2:str):
    line = cim.ACLineSegment(name = name, mRID = str(uuid.uuid4()), EquipmentContainer=network.Feeder)
    node1 = cim.ConnectivityNode(name = bus1, mRID = str(uuid.uuid4()), ConnectivityNodeContainer=network.Feeder)
    node2 = cim.ConnectivityNode(name = bus2, mRID = str(uuid.uuid4()), ConnectivityNodeContainer=network.Feeder)
    term1 = cim.Terminal(mRID = str(uuid.uuid4()))
    term2 = cim.Terminal(mRID = str(uuid.uuid4()))

    term1.ConnectivityNode = node1
    term2.ConnectivityNode = node2
    term1.ConductingEquipment = line
    term2.ConductingEquipment = line
    node1.Terminals.append(term1)
    node2.Terminals.append(term2)
    line.Terminals.append(term1)
    line.Terminals.append(term2)

    network.add_to_typed_catalog([line, node1, node2, term1, term2])

In [8]:
name = 'line_1'
bus1 = 'bus_1'
bus2 = 'bus_2'

create_line(network, name, bus1, bus2)

for line in network.typed_catalog[cim.ACLineSegment].values():
    line.r = 0.004
    line.x = 0.080
    line.b = 0.006

## Export and Upload

In [ ]:
network.write_xml('demo_feeder.xml', "http://iec.ch/TC57/CIM100#")

In [ ]:
network.

In [ ]:
network.upload()

In [ ]:
!python3 /home/ande188/CIMHub/src_python/cimhub/ListFeeders.py

In [ ]:
cim.Measurement.PowerSystemResource
cim.ACLineSegment.

In [ ]:
cim.LinearShuntCompensator().ShuntCompensatorPhase

In [ ]:
cim.ACLineSegment().ACLineSegmentPhases